In [1]:
import torch
from transformers import AutoTokenizer, LlamaForCausalLM

/home/james/anaconda3/envs/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {str(device).upper()}")

Device: CUDA


In [3]:
model_dir = "/mnt/artemis/library/weights/meta/llama-2/7Bf/hf"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
print(tokenizer.bos_token)
print(tokenizer.eos_token)

<s>
</s>


In [5]:
model = LlamaForCausalLM.from_pretrained(
    model_dir,
    torch_dtype = torch.float16,
    device_map="auto",
    )

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]


In [6]:
en1 = "Prometheus stole fire from the gods and gave it to man."
tok = tokenizer(en1, return_tensors='pt').to(device).input_ids
print(tok)

tensor([[    1,  9705, 23043,   375,   380,  1772,  3974,   515,   278, 27379,
           322,  4846,   372,   304,   767, 29889]], device='cuda:0')


In [8]:
tokenizer.batch_decode(tok, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]

'<s> Prometheus stole fire from the gods and gave it to man.'

In [9]:
while 1:
    tok = model.generate(
        input_ids=tok,
        max_new_tokens = 1,
        do_sample = True,
        top_p = 1.0,
        top_k = 10,
        temperature = 0.7,
        pad_token_id = tokenizer.eos_token_id,
        repetition_penalty = 1.0,
        length_penalty = 1
        )

    output = tok[0,-1].item()

    if tokenizer.decode(torch.tensor(output)) == tokenizer.eos_token:
        break
    else:
        response = tokenizer.batch_decode(tok, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]
        print(response)

<s> Prometheus stole fire from the gods and gave it to man. In
<s> Prometheus stole fire from the gods and gave it to man. In the
<s> Prometheus stole fire from the gods and gave it to man. In the Greek
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth,
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth, Prom
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth, Promethe
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth, Prometheus
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth, Prometheus was
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth, Prometheus was a
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth, Prometheus was a tit
<s> Prometheus stole fire from the gods and gave it to man. In the Greek myth, Prometheu